# replace this dataset with the original piotr homography keypoint, then see if adding to it is helpful we corrupted the ultimate with bad data

In [1]:
# %%writefile src/computer_vision/download_dataset.py
# !pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="htpcxp3XQh7SsgMfjJns")
project = rf.workspace("basketball-formations").project("ultimate_basketball_court_keypoint")
version = project.version(1)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...


KeyboardInterrupt: 

In [ ]:
# %%writefile src/computer_vision/train_court_keypoints.py

#!/usr/bin/env python3
"""
train_court_keypoints.py

A simplified Python module to train a YOLOv8 pose (keypoint) model on the Roboflow
"ultimate_basketball_court_keypoint" dataset without argparse. Steps:
 1. Download dataset from Roboflow using standard download (no custom location)
 2. Move dataset folder to data/images/ preserving structure
 3. Patch data.yaml to point to local train/valid/test images
 4. Train using Ultralytics YOLOv8 pose
 5. Validate model
 6. Run inference on an example image
 7. (Optional) Deploy to Roboflow
 8. (Optional) Clean up dataset files

Usage:
    Adjust variables in the __main__ block and run:
        python train_court_keypoints.py

Requires: roboflow, ultralytics
"""
from roboflow import Roboflow
import numpy as np
import cv2
import shutil
import sys, subprocess
from ultralytics import YOLO
from pathlib import Path
import matplotlib.pyplot as plt


def download_and_stage_dataset(api_key: str,
                               workspace: str,
                               project: str,
                               version: int,
                               images_root: Path) -> Path:
    """
    Skip download if the dataset is already staged and complete.
    Otherwise perform a standard Roboflow download, move it under images_root,
    and return the path.
    """
    images_root.mkdir(parents=True, exist_ok=True)
    dst_path = images_root / f"{project.replace('/', '_')}-{version}"

    expected_dirs = [
        dst_path / "train" / "images",
        dst_path / "valid" / "images",
        dst_path / "test"  / "images",
    ]
    if all(p.exists() for p in expected_dirs):
        print(f"🟢 Dataset already present at {dst_path} – skipping download.")
        return dst_path

    # dataset missing or corrupted → download fresh
    print("🔄 Dataset not found or incomplete – downloading from Roboflow …")
    rf = Roboflow(api_key=api_key)
    ds = rf.workspace(workspace).project(project).version(version)
    dataset = ds.download(model_format="yolov8")  # standard download
    src_path = Path(dataset.location)             # e.g. Ultimate_basketball_court_keypoint-1

    # remove old remnants if any
    if dst_path.exists():
        shutil.rmtree(dst_path)
    shutil.move(src_path.as_posix(), dst_path.as_posix())
    print(f"📁 Dataset staged at {dst_path}")
    return dst_path



def patch_data_yaml(dataset_path: Path) -> str:
    """
    Patch data.yaml paths so that YOLOv8 sees local train/valid/test images
    inside the same dataset root.
    """
    yaml_path = dataset_path / "data.yaml"
    if not yaml_path.exists():
        raise FileNotFoundError(f"data.yaml not found at {yaml_path}")

    txt = yaml_path.read_text()
    
    # Handle Roboflow's default "../" prefixes and ensure correct relative paths
    for key in ("train:", "val:", "test:"):
        if key in txt:
            if "../" in txt:
                # Strip leading ../ from Roboflow default paths
                txt = txt.replace(f"{key} ../", f"{key} ")
            # Ensure paths point to correct subdirectories
            if key == "train:" and "train/images" not in txt:
                txt = txt.replace(f"{key} train", f"{key} train/images")
            elif key == "val:" and "valid/images" not in txt:
                txt = txt.replace(f"{key} valid", f"{key} valid/images")
            elif key == "test:" and "test/images" not in txt:
                txt = txt.replace(f"{key} test", f"{key} test/images")
    
    yaml_path.write_text(txt)
    print(f"Patched data.yaml paths in {yaml_path}")
    return yaml_path.as_posix()


def check_gpu():
    """Ensure CUDA GPU is available via `nvidia-smi`."""
    try:
        subprocess.run(["nvidia-smi"], check=True, stdout=subprocess.DEVNULL)
        print("✅ GPU detected.")
    except subprocess.CalledProcessError:
        sys.exit("❌ No GPU found. Please enable CUDA or switch to a GPU runtime.")


def train_pose_model(
    data_yaml: str,
    model_checkpoint: str,
    epochs: int,
    batch: int,
    imgsz: int,
    device: int,
    project_dir: str,
    run_name: str,
    mosaic: float = 0.0,
    plots: bool = True,
    lr0: float = 1e-3
) -> Path:
    """
    Wrapper around Ultralytics train() with safe scale handling.
    """
    model = YOLO(model_checkpoint)
    print(f"Starting training: epochs={epochs}, batch={batch}, imgsz={imgsz}, "
          f"mosaic={mosaic}, plots={plots}")

    model.train(
        task    = "pose",
        data    = data_yaml,
        epochs  = epochs,
        batch   = batch,
        imgsz   = imgsz,
        lr0     = lr0,
        device  = device,
        mosaic  = mosaic,
        plots   = plots,
        project = project_dir,
        name    = run_name
    )
    best_weights = Path(project_dir) / run_name / "weights" / "best.pt"
    print(f"✅ Training complete. Best weights at {best_weights}")
    return best_weights



def validate_model(weights_path: Path, data_yaml: str, imgsz: int, batch: int, device: int):
    """Run YOLOv8 validation on the trained weights."""
    print("Validating model...")
    cmd = [
        "yolo", "task=pose", "mode=val",
        f"model={weights_path}",
        f"data={data_yaml}",
        f"imgsz={imgsz}",
        f"batch={batch}",
        f"device={device}"
    ]
    subprocess.run(cmd, check=True)
    print("✅ Validation finished.")




def run_inference(model: YOLO, source: str, save_dir: str,
                  conf: float, imgsz: int, device: int):
    """Run inference and disable letterbox padding (rect=True)."""
    results = model.predict(
        source=source, save=True, conf=conf, imgsz=imgsz,
        device=device, project=save_dir, name="inference",
        rect=True
    )
    return results[0]

def filter_keypoints(keypoints, conf_thresh: float = 0.5):
    """Return source keypoints & mask for confidences > conf_thresh."""
    confidences = keypoints.confidence[0]
    mask = confidences > conf_thresh
    source_pts = keypoints.xy[0][mask]
    return source_pts.astype(np.float32), mask

def compute_homography(source: np.ndarray, target: np.ndarray,
                       ransac_thresh: float = 3.0):
    """Compute H with RANSAC. Returns H matrix & inlier mask."""
    H, inlier_mask = cv2.findHomography(
        source, target, cv2.RANSAC, ransacReprojThreshold=ransac_thresh
    )
    if H is None:
        raise ValueError("Homography computation failed.")
    return H, inlier_mask.ravel().astype(bool)

def undistort_image(img: np.ndarray,
                    camera_matrix: np.ndarray,
                    dist_coeffs: np.ndarray):
    """Undistort images to correct lens distortion."""
    return cv2.undistort(img, camera_matrix, dist_coeffs, None, None)

def visualize_matches(source, target, inliers):
    """Plot source vs. target, green=inlier, red=outlier."""
    plt.figure(figsize=(6,6))
    for (x,y), (u,v), ok in zip(source, target, inliers):
        c = 'g' if ok else 'r'
        plt.scatter(x, y, marker='x', color=c)
        plt.scatter(u, v, marker='o', color=c)
    plt.title(f"Inliers: {inliers.sum()}, Outliers: {len(inliers)-inliers.sum()}")
    plt.show()



def deploy_to_roboflow(project, dataset_version: int, model_path: Path):
    """Deploy the trained model back to Roboflow."""
    print("Deploying model to Roboflow...")
    project.version(dataset_version).deploy(
        model_type="yolov8-pose",
        model_path=str(model_path)
    )
    print("✅ Deployment complete.")


if __name__ == "__main__":
    # === User configuration ===
    API_KEY = "htpcxp3XQh7SsgMfjJns"
    WORKSPACE = "basketball-formations"
    PROJECT_NAME = "ultimate_basketball_court_keypoint"
    DATASET_VERSION = 1
    IMAGES_DIR = Path("data/images")  # No dataset name yet - will be added

    EPOCHS = 100
    BATCH = 16
    IMG_SIZE = 640
    MODEL_CHECKPOINT = "data/models/yolov8x-pose.pt"
    DEVICE = 0            # CUDA device index
    mosaic=0.0
    plots=True
    INFERENCE_IMAGE = None         # e.g. "court_sample.jpg"
    DEPLOY = False        # set True to deploy model
    CLEANUP_AFTER_TRAIN = True      # set True to delete dataset after training
    # =============================

    check_gpu()

    # Download dataset using standard Roboflow download and move to images directory
    dataset_path = download_and_stage_dataset(
        API_KEY, WORKSPACE, PROJECT_NAME, DATASET_VERSION, IMAGES_DIR
    )
    
    # Patch YAML to use correct relative paths
    data_yaml = patch_data_yaml(dataset_path)

    # Train the model
    run_name = f"basketball_court_kpts_v{DATASET_VERSION}"
    best_weights = train_pose_model(
        data_yaml,
        MODEL_CHECKPOINT,
        EPOCHS,
        BATCH,
        IMG_SIZE,
        DEVICE,
        project_dir="runs/pose",
        run_name=run_name,
        mosaic=mosaic,
        plots=True,
        # scale=(0.5,1.0),
        lr0=1e-3
    )

    # Validate the trained model
    validate_model(
        best_weights,
        data_yaml,
        IMG_SIZE,
        BATCH,
        DEVICE
    )

    # Optional inference
    model = YOLO(best_weights)
    if INFERENCE_IMAGE:
        run_inference(
            model,
            INFERENCE_IMAGE,
            save_dir="runs",
            conf=0.25,
            imgsz=IMG_SIZE,
            device=DEVICE
        )

    # Optional deployment to Roboflow
    if DEPLOY:
        rf = Roboflow(api_key=API_KEY)
        project = rf.workspace(WORKSPACE).project(PROJECT_NAME)
        deploy_to_roboflow(project, DATASET_VERSION, best_weights)

    # Optional cleanup - remove dataset to free disk space
    if CLEANUP_AFTER_TRAIN:
        print(f"🧹 Deleting staged dataset at {dataset_path}")
        shutil.rmtree(dataset_path)
        print("✅ Cleanup complete.")


✅ GPU detected.
🟢 Dataset already present at data\images\ultimate_basketball_court_keypoint-1 – skipping download.
Patched data.yaml paths in data\images\ultimate_basketball_court_keypoint-1\data.yaml
Starting training: epochs=100, batch=16, imgsz=640, mosaic=0.0, plots=True
New https://pypi.org/project/ultralytics/8.3.159 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.158  Python-3.10.17 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/images/ultimate_basketball_court_keypoint-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze

train: Scanning C:\Users\ghadf\vscode_projects\docker_projects\bball_homography\data\images\Ultimate_basketball_court_keypoint-1\train\labels.cache... 1986 images, 0 backgrounds, 12 corrupt: 100%|██████████| 1986/1986 [00:00<?, ?it/s]

train: C:\Users\ghadf\vscode_projects\docker_projects\bball_homography\data\images\Ultimate_basketball_court_keypoint-1\train\images\yt-9SjvZPFiDH0-0009_jpg.rf.3b8ca70fb283f14e5d2b3888b765cc02.jpg: ignoring corrupt image/label: negative label values [ -0.0022322]
train: C:\Users\ghadf\vscode_projects\docker_projects\bball_homography\data\images\Ultimate_basketball_court_keypoint-1\train\images\yt-9SjvZPFiDH0-0009_jpg.rf.a92a7a74647d8f2ba9224f11a4ba6211.jpg: ignoring corrupt image/label: negative label values [ -0.0022322]
train: C:\Users\ghadf\vscode_projects\docker_projects\bball_homography\data\images\Ultimate_basketball_court_keypoint-1\train\images\yt-9SjvZPFiDH0-0009_jpg.rf.d39777884d95475a73d77a0b94809ce9.jpg: ignoring corrupt image/label: negative label values [ -0.0022322]
train: C:\Users\ghadf\vscode_projects\docker_projects\bball_homography\data\images\Ultimate_basketball_court_keypoint-1\train\images\yt-SbbnJhBb67k-0018_jpg.rf.8fcd41d393483d8ed4210afe7b14bc24.jpg: ignoring c

val: Fast image access  (ping: 0.10.1 ms, read: 480.3120.1 MB/s, size: 85.2 KB)


val: Scanning C:\Users\ghadf\vscode_projects\docker_projects\bball_homography\data\images\Ultimate_basketball_court_keypoint-1\valid\labels.cache... 165 images, 0 backgrounds, 1 corrupt: 100%|██████████| 165/165 [00:00<?, ?it/s]

...

    100/100      13.9G     0.2105      4.856     0.1748     0.1454     0.9887          6        640: 100%|██████████| 124/124 [00:37<00:00,  3.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  3.69it/s]

                   all        164        164      0.998       0.99      0.992      0.878      0.761      0.756      0.718      0.215



100 epochs completed in 1.152 hours.
Optimizer stripped from runs\pose\basketball_court_kpts_v1\weights\last.pt, 140.2MB
Optimizer stripped from runs\pose\basketball_court_kpts_v1\weights\best.pt, 140.2MB

Validating runs\pose\basketball_court_kpts_v1\weights\best.pt...
Ultralytics 8.3.158  Python-3.10.17 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLOv8x-pose summary (fused): 121 layers, 69,846,015 parameters, 0 gradients, 265.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.87it/s]


                   all        164        164      0.998       0.99      0.992      0.885      0.771      0.766      0.721      0.213
                 court        113        113      0.998          1      0.995      0.956      0.883      0.885       0.88      0.324
            full_court         51         51      0.998       0.98      0.989      0.814      0.658      0.647      0.561      0.101
Speed: 0.8ms preprocess, 5.7ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs\pose\basketball_court_kpts_v1
✅ Training complete. Best weights at runs\pose\basketball_court_kpts_v1\weights\best.pt
Validating model...
✅ Validation finished.
🧹 Deleting staged dataset at data\images\ultimate_basketball_court_keypoint-1
✅ Cleanup complete.


In [ ]:
from pathlib import Path
import subprocess
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys

# === Configuration ===
# Relative path to the run directory you want to review
RUNS_DIR = Path("runs") / "pose" / "basketball_court_kpts_v1"

# Relative path to your data.yaml file
# Ensure this matches the actual folder name (case-sensitive)
DATASET_YAML = Path("data") / "images" / "Ultimate_basketball_court_keypoint-1" / "data.yaml"

# Exit if data.yaml is missing
if not DATASET_YAML.exists():
    print(f"[!] data.yaml not found at {DATASET_YAML}. Please verify the path.")
    sys.exit(1)


def list_run_results(run_dir: Path):
    """Prints the contents of the run directory."""
    print(f"Contents of {run_dir}:")
    if not run_dir.exists():
        print(f"[!] Run directory not found: {run_dir}")
        return
    for f in sorted(run_dir.iterdir()):
        print(f" - {f.name}")


def display_image(image_path: Path, figsize=(8, 8)):
    """Displays an image if it exists, otherwise prints a warning."""
    if image_path.exists():
        img = mpimg.imread(str(image_path))
        plt.figure(figsize=figsize)
        plt.imshow(img)
        plt.axis('off')
        plt.title(image_path.name)
        plt.show()
    else:
        print(f"[!] Image not found: {image_path}")


def validate_model(run_weights: Path, data_yaml: Path):
    """Runs YOLOv8 validation via subprocess with robust error handling."""
    weights_path = run_weights.resolve().as_posix()
    yaml_path    = data_yaml.resolve().as_posix()
    cmd = [
        "yolo", "task=pose", "mode=val",
        f"model={weights_path}",
        f"data={yaml_path}"
    ]
    print(f"Running validation: {' '.join(cmd)}")
    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print("✅ Validation succeeded.")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"❌ Validation failed with exit code {e.returncode}.")
        if e.stdout:
            print("-- STDOUT --")
            print(e.stdout)
        if e.stderr:
            print("-- STDERR --")
            print(e.stderr)


def main():
    # 1. List all files in the run directory
    list_run_results(RUNS_DIR)

    # 2. Display key result images
    display_image(RUNS_DIR / "results.png")
    display_image(RUNS_DIR / "val_batch0_pred.jpg")

    # 3. Validate the best model
    best_weights = RUNS_DIR / "weights" / "best.pt"
    if not best_weights.exists():
        print(f"[!] Best weights not found at {best_weights}")
    else:
        validate_model(best_weights, DATASET_YAML)


if __name__ == "__main__":
    main()


Contents of runs\pose\basketball_court_kpts_v1:
 - args.yaml
 - confusion_matrix.png
 - confusion_matrix_normalized.png
 - labels.jpg
 - labels_correlogram.jpg
 - PoseF1_curve.png
 - PoseP_curve.png
 - PosePR_curve.png
 - PoseR_curve.png
 - results.csv
 - results.png
 - train_batch0.jpg
 - train_batch1.jpg
 - train_batch11160.jpg
 - train_batch11161.jpg
 - train_batch11162.jpg
 - train_batch2.jpg
 - val_batch0_labels.jpg
 - val_batch0_pred.jpg
 - val_batch1_labels.jpg
 - val_batch1_pred.jpg
 - val_batch2_labels.jpg
 - val_batch2_pred.jpg
 - weights


<Figure size 800x800 with 1 Axes>

<Figure size 800x800 with 1 Axes>

Running validation: yolo task=pose mode=val model=C:/Users/ghadf/vscode_projects/docker_projects/bball_homography/runs/pose/basketball_court_kpts_v1/weights/best.pt data=C:/Users/ghadf/vscode_projects/docker_projects/bball_homography/data/images/Ultimate_basketball_court_keypoint-1/data.yaml


Exception in thread Thread-42 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\ghadf\AppData\Roaming\uv\python\cpython-3.10.17-windows-x86_64-none\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\ghadf\vscode_projects\docker_projects\bball_homography\.venv\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\ghadf\AppData\Roaming\uv\python\cpython-3.10.17-windows-x86_64-none\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ghadf\AppData\Roaming\uv\python\cpython-3.10.17-windows-x86_64-none\lib\subprocess.py", line 1515, in _readerthread
    buffer.append(fh.read())
  File "C:\Users\ghadf\AppData\Roaming\uv\python\cpython-3.10.17-windows-x86_64-none\lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in 

✅ Validation succeeded.
Ultralytics 8.3.158  Python-3.10.17 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLOv8x-pose summary (fused): 121 layers, 69,846,015 parameters, 0 gradients, 265.0 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 22.35.0 MB/s, size: 86.2 KB)
val: C:\Users\ghadf\vscode_projects\docker_projects\bball_homography\data\images\Ultimate_basketball_court_keypoint-1\valid\images\yt-eLF46VmxXxI-0001_jpg.rf.9a8cdc1409cf168f37811de710d1bbde.jpg: ignoring corrupt image/label: negative label values [ -0.0066964  -0.0066964  -0.0066964]
val: New cache created: C:\Users\ghadf\vscode_projects\docker_projects\bball_homography\data\images\Ultimate_basketball_court_keypoint-1\valid\labels.cache
                   all        164        164      0.998       0.99      0.992      0.884      0.771      0.766       0.72      0.212
                 court        113        113      0.998          1      0.995      0.957      0.883      0.885       0.88      0.323
   